In [1]:
!pip install transformers[torch]
!pip install sentencepiece evaluate --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 14.9 MB/s eta 0:00:00


In [2]:
import pandas as pd
import torch
import numpy as np
from transformers import XLNetTokenizer, XLNetForSequenceClassification, Trainer, TrainingArguments

In [3]:
X_train = pd.read_csv('https://raw.githubusercontent.com/psochando/sanchez.github.io/main/X_train_5000_mayus.csv')['Cuerpo']
X_test = pd.read_csv('https://raw.githubusercontent.com/psochando/sanchez.github.io/main/X_test_5000_mayus.csv')['Cuerpo']
y_train = pd.read_csv('https://raw.githubusercontent.com/psochando/sanchez.github.io/main/y_train_5000.csv')['Periódico']
y_test = pd.read_csv('https://raw.githubusercontent.com/psochando/sanchez.github.io/main/y_test_5000.csv')['Periódico']

In [4]:
label_map = {label: index for index, label in enumerate(np.unique(y_train))}
y_train = y_train.map(label_map)
y_test = y_test.map(label_map)

In [5]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)

cuda:0


In [6]:
X_test

0       El mercado inmobiliario es, a veces, mucho más...
1       Aaron Donald es el mejor defensor de la última...
2       Don Mariano Rajoy Sobredo, padre del president...
3       Lío importante anoche en el estadio Santiago B...
4       Con huelga o sin huelga de caseteros, habrá Fe...
                              ...                        
1489    Se llama Optimus. Mide 1,73. Pesa 57 kilos. Ca...
1490    Si la taxonomía verde es un indicador, el año ...
1491    Nada más comprobar que la moción de censura pr...
1492    El Gobierno conservador del Reino Unido sabe q...
1493    La Consejería de Economía, Hacienda y Empleo h...
Name: Cuerpo, Length: 1494, dtype: object

In [7]:
num_classes = len(y_train.unique())

In [8]:
tokenizer = XLNetTokenizer.from_pretrained('xlnet-large-cased')

In [9]:
train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True, max_length = 2000)
test_encodings = tokenizer(X_test.tolist(), truncation=True, padding=True, max_length = 2000)

In [10]:
# Crear el dataset personalizado para PyTorch
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {
            'input_ids': torch.tensor(self.encodings['input_ids'][idx]),
            'attention_mask': torch.tensor(self.encodings['attention_mask'][idx]),
            'labels': torch.tensor(self.labels[idx])
        }
        return item

    def __len__(self):
        return len(self.labels)

In [11]:
train_dataset = MyDataset(train_encodings, y_train)
test_dataset = MyDataset(test_encodings, y_test)


In [12]:
def compute_metrics(pred):
    logits, labels = pred
    predictions = np.argmax(logits, axis=1)
    return {"accuracy": (predictions == labels).mean()}


In [13]:
# Configurar los argumentos de entrenamiento
training_args = TrainingArguments(
    per_device_train_batch_size=2,
    per_device_eval_batch_size=48,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    output_dir="./results"
)

In [14]:
model = XLNetForSequenceClassification.from_pretrained('xlnet-large-cased', num_labels=num_classes).to(device)

Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-large-cased and are newly initialized: ['logits_proj.weight', 'sequence_summary.summary.weight', 'logits_proj.bias', 'sequence_summary.summary.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
# Crear el entrenador y entrenar el modelo
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

In [16]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


OutOfMemoryError: ignored

In [ ]:
# test_preds = []
# for i in range(len(X_test)):
#     val_encoding = tokenizer(X_test.iloc[i], truncation=True, padding=True, return_tensors="pt").to(device)
#     outputs = model(**val_encoding)
#     logits = outputs.logits.cpu().detach().numpy()
#     test_preds.append(np.argmax(logits))

In [ ]:
print('accuracy del train:')
trainer.evaluate(train_dataset)['eval_accuracy']

In [ ]:
print('accuracy del test:')
trainer.evaluate(test_dataset)['eval_accuracy']

In [ ]:
trainer.save_model("./XLNET_large_1_mayus")